In [9]:
#!wget https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
#!tar -xf aclImdb_v1.tar.gz

In [47]:
import tensorflow as tf
import keras

dataset = keras.utils.text_dataset_from_directory(directory='aclImdb', label_mode=None, batch_size=256)
dataset = dataset.map(lambda x: tf.strings.regex_replace(x, '<br />', ' '))

Found 100006 files belonging to 1 classes.


In [48]:
from keras import layers

seq_len = 100
vocab_size = 15000

text_vec = layers.TextVectorization(max_tokens=vocab_size, output_mode='int', output_sequence_length=seq_len)
text_vec.adapt(dataset)

In [49]:
def prepare_lm_dataset(text_batch):
    vectorized_seq = text_vec(text_batch)
    x = vectorized_seq[:, :-1]
    y = vectorized_seq[:, 1:]
    return x, y

lm_dataset = dataset.map(prepare_lm_dataset, num_parallel_calls=16)

for d in lm_dataset:
    for a in d:
        print('-')
        print(a)
    break

-
tf.Tensor(
[[ 378    7    4 ...   26  569   35]
 [  36    8   60 ... 2057    8   29]
 [1392 4126   16 ...    4  180  395]
 ...
 [   1    7   11 ...   11  494    3]
 [  10 1205  717 ...  422 2971    5]
 [  11   13    4 ... 5668    5   46]], shape=(256, 99), dtype=int64)
-
tf.Tensor(
[[   7    4 1923 ...  569   35   24]
 [   8   60  203 ...    8   29  547]
 [4126   16    2 ...  180  395    3]
 ...
 [   7   11 2199 ...  494    3 1064]
 [1205  717   11 ... 2971    5    2]
 [  13    4  651 ...    5   46    5]], shape=(256, 99), dtype=int64)


In [50]:
class TransformerEncoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim)
        self.dense_proj = keras.Sequential(
            [layers.Dense(dense_dim, activation="relu"),
             layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()

    def call(self, inputs, mask=None):
        if mask is not None:
            mask = mask[:, tf.newaxis, :]
        attention_output = self.attention(
            inputs, inputs, attention_mask=mask)
        proj_input = self.layernorm_1(inputs + attention_output)
        proj_output = self.dense_proj(proj_input)
        return self.layernorm_2(proj_input + proj_output)

    def get_config(self):
        config = super().get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "num_heads": self.num_heads,
            "dense_dim": self.dense_dim,
        })
        return config


class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, input_dim, output_dim, **kwargs):
        super().__init__(**kwargs)
        self.token_embeddings = layers.Embedding(
            input_dim=input_dim, output_dim=output_dim)
        self.position_embeddings = layers.Embedding(
            input_dim=sequence_length, output_dim=output_dim)
        self.sequence_length = sequence_length
        self.input_dim = input_dim
        self.output_dim = output_dim

    def call(self, inputs):
        length = tf.shape(inputs)[-1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_tokens = self.token_embeddings(inputs)
        embedded_positions = self.position_embeddings(positions)
        return embedded_tokens + embedded_positions

    def compute_mask(self, inputs, mask=None):
        return tf.math.not_equal(inputs, 0)

    def get_config(self):
        config = super().get_config()
        config.update({
            "output_dim": self.output_dim,
            "sequence_length": self.sequence_length,
            "input_dim": self.input_dim,
        })
        return config


class TransformerDecoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention_1 = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.attention_2 = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.dense_proj = keras.Sequential([
            layers.Dense(dense_dim, activation=activations.relu),
            layers.Dense(embed_dim)
        ])
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.layernorm_3 = layers.LayerNormalization()

    def get_config(self):
        config = super().get_config()
        config.update({
            'embed_dim': self.embed_dim,
            'dense_dim': self.dense_dim,
            'num_heads': self.num_heads
        })
        return config
    
    def get_casual_attention_mask(self, inputs):
        input_shape = tf.shape(inputs)
        batch_size, seq_len = input_shape[0], input_shape[1]
        i = tf.range(seq_len)[:, tf.newaxis]
        j = tf.range(seq_len)
        mask = tf.cast(i >= j, dtype=tf.int32)
        mask = tf.reshape(mask, (1, input_shape[1], input_shape[1]))
        mult = tf.concat([
            tf.expand_dims(batch_size, -1),
            tf.constant([1, 1], dtype=tf.int32)
        ], axis=0)
        return tf.tile(mask, mult)
    
    def call(self, inputs, encoder_outputs, mask=None):
        casual_mask = self.get_casual_attention_mask(inputs)
        if mask is not None:
            padding_mask = tf.cast(mask[:, tf.newaxis, :], dtype=tf.int32)
            padding_mask = tf.minimum(padding_mask, casual_mask)
        attention_output_1 = self.attention_1(
            query=inputs,
            value=inputs,
            key=inputs,
            attention_mask=casual_mask
        )
        attention_output_1 = self.layernorm_1(inputs + attention_output_1)
        attention_output_2 = self.attention_2(
            query=attention_output_1,
            value=encoder_outputs,
            key=encoder_outputs,
            attention_mask=padding_mask
        )
        attention_output_2 = self.layernorm_2(attention_output_1 + attention_output_2)
        proj_output = self.dense_proj(attention_output_2)
        return self.layernorm_3(attention_output_2 + proj_output)

In [51]:
from keras import activations, losses, optimizers

embed_dim = 256
latent_dim = 2048
num_heads = 2

inputs = layers.Input(shape=(None,), dtype=tf.int64)
x = PositionalEmbedding(sequence_length=seq_len, input_dim=vocab_size, output_dim=embed_dim)(inputs)
x = TransformerDecoder(embed_dim=embed_dim, dense_dim=latent_dim, num_heads=num_heads)(x, x)
outputs = layers.Dense(vocab_size, activation=activations.softmax)(x)
model = keras.Model(inputs, outputs)
model.compile(loss=losses.SparseCategoricalCrossentropy(), optimizer=optimizers.RMSprop())

In [52]:
import numpy as np

token_index = dict(enumerate(text_vec.get_vocabulary()))

def sample_text(pred, temp=1.0):
    pred = np.asarray(pred).astype(np.float64)
    pred = np.log(pred) / temp
    exp_preds = np.exp(pred)
    pred = exp_preds / np.sum(exp_preds)
    prob = np.random.multinomial(n=1, pvals=pred, size=1)
    return np.argmax(prob)

class TextGenerator(keras.callbacks.Callback):
    def __init__(self, prompt, gen_len, temps=(1.0,), print_freq=1):
        self.prompt = prompt
        self.gen_len = gen_len
        self.temps = temps
        self.print_freq = print_freq

    def on_epoch_end(self, epoch, logs=None):
        if (epoch + 1) % self.print_freq != 0:
            return
        
        for temp in self.temps:
            print(f'Temperature = {temp}:')
            sentence = self.prompt
            for i in range(self.gen_len):
                tokenized_sentence = text_vec([sentence])
                pred = self.model(tokenized_sentence)
                next_token = sample_text(pred=pred[0, i, :], temp=temp)
                sampled_token = token_index[next_token]
                sentence += ' ' + sampled_token
            print(sentence)

prompt = 'This movie'
text_gen_callback = TextGenerator(prompt=prompt, gen_len=50, temps=(0.2, 0.5, 0.7, 1, 1.5))

In [53]:
model.fit(lm_dataset, epochs=1, callbacks=[text_gen_callback])

391/391 [==============================] - ETA: 0s - loss: 5.7080Temperature = 0.2:
This movie is is a a good movie movie that i it have is been a a good good movie movie that i it thought is it a was [UNK] a and good i movie have i been was a the very movie good i movie was i a was [UNK] a
Temperature = 0.5:
This movie film is is a a [UNK] good [UNK] movie the but original for series the i first have good no movie idea is the a film [UNK] it [UNK] was it a is very a well movie the i plot was is the a main lot characters of were the
Temperature = 0.7:
This movie is is somewhat a the truly acting a film great that movie are is not that only i to created be to a the good film the that [UNK] is and the a only very thing good is reason a to bad the people [UNK] in experiment the because movie
Temperature = 1:
This movie is [UNK] bad and women thought who that but make i trash just is not pretty saying allowed amazing to i 10 love years the after level all but time it it have off been but s

In [46]:
import tensorflow as tf
from tensorflow import keras
dataset = keras.utils.text_dataset_from_directory(
    directory="aclImdb", label_mode=None, batch_size=256)
dataset = dataset.map(lambda x: tf.strings.regex_replace(x, "<br />", " "))

from tensorflow.keras.layers import TextVectorization

sequence_length = 100
vocab_size = 15000
text_vectorization = TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length,
)
text_vectorization.adapt(dataset)

def prepare_lm_dataset(text_batch):
    vectorized_sequences = text_vectorization(text_batch)
    x = vectorized_sequences[:, :-1]
    y = vectorized_sequences[:, 1:]
    return x, y

lm_dataset = dataset.map(prepare_lm_dataset, num_parallel_calls=4)

import tensorflow as tf
from tensorflow.keras import layers

class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, input_dim, output_dim, **kwargs):
        super().__init__(**kwargs)
        self.token_embeddings = layers.Embedding(
            input_dim=input_dim, output_dim=output_dim)
        self.position_embeddings = layers.Embedding(
            input_dim=sequence_length, output_dim=output_dim)
        self.sequence_length = sequence_length
        self.input_dim = input_dim
        self.output_dim = output_dim

    def call(self, inputs):
        length = tf.shape(inputs)[-1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_tokens = self.token_embeddings(inputs)
        embedded_positions = self.position_embeddings(positions)
        return embedded_tokens + embedded_positions

    def compute_mask(self, inputs, mask=None):
        return tf.math.not_equal(inputs, 0)

    def get_config(self):
        config = super(PositionalEmbedding, self).get_config()
        config.update({
            "output_dim": self.output_dim,
            "sequence_length": self.sequence_length,
            "input_dim": self.input_dim,
        })
        return config


class TransformerDecoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention_1 = layers.MultiHeadAttention(
          num_heads=num_heads, key_dim=embed_dim)
        self.attention_2 = layers.MultiHeadAttention(
          num_heads=num_heads, key_dim=embed_dim)
        self.dense_proj = keras.Sequential(
            [layers.Dense(dense_dim, activation="relu"),
             layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.layernorm_3 = layers.LayerNormalization()
        self.supports_masking = True

    def get_config(self):
        config = super(TransformerDecoder, self).get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "num_heads": self.num_heads,
            "dense_dim": self.dense_dim,
        })
        return config

    def get_causal_attention_mask(self, inputs):
        input_shape = tf.shape(inputs)
        batch_size, sequence_length = input_shape[0], input_shape[1]
        i = tf.range(sequence_length)[:, tf.newaxis]
        j = tf.range(sequence_length)
        mask = tf.cast(i >= j, dtype="int32")
        mask = tf.reshape(mask, (1, input_shape[1], input_shape[1]))
        mult = tf.concat(
            [tf.expand_dims(batch_size, -1),
             tf.constant([1, 1], dtype=tf.int32)], axis=0)
        return tf.tile(mask, mult)

    def call(self, inputs, encoder_outputs, mask=None):
        causal_mask = self.get_causal_attention_mask(inputs)
        if mask is not None:
            padding_mask = tf.cast(
                mask[:, tf.newaxis, :], dtype="int32")
            padding_mask = tf.minimum(padding_mask, causal_mask)
        else:
            padding_mask = mask
        attention_output_1 = self.attention_1(
            query=inputs,
            value=inputs,
            key=inputs,
            attention_mask=causal_mask)
        attention_output_1 = self.layernorm_1(inputs + attention_output_1)
        attention_output_2 = self.attention_2(
            query=attention_output_1,
            value=encoder_outputs,
            key=encoder_outputs,
            attention_mask=padding_mask,
        )
        attention_output_2 = self.layernorm_2(
            attention_output_1 + attention_output_2)
        proj_output = self.dense_proj(attention_output_2)
        return self.layernorm_3(attention_output_2 + proj_output)


from tensorflow.keras import layers
embed_dim = 256
latent_dim = 2048
num_heads = 2

inputs = keras.Input(shape=(None,), dtype="int64")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(inputs)
x = TransformerDecoder(embed_dim, latent_dim, num_heads)(x, x)
outputs = layers.Dense(vocab_size, activation="softmax")(x)
model = keras.Model(inputs, outputs)
model.compile(loss="sparse_categorical_crossentropy", optimizer="rmsprop")


import numpy as np

tokens_index = dict(enumerate(text_vectorization.get_vocabulary()))

def sample_next(predictions, temperature=1.0):
    predictions = np.asarray(predictions).astype("float64")
    predictions = np.log(predictions) / temperature
    exp_preds = np.exp(predictions)
    predictions = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, predictions, 1)
    return np.argmax(probas)

class TextGenerator(keras.callbacks.Callback):
    def __init__(self,
                 prompt,
                 generate_length,
                 model_input_length,
                 temperatures=(1.,),
                 print_freq=1):
        self.prompt = prompt
        self.generate_length = generate_length
        self.model_input_length = model_input_length
        self.temperatures = temperatures
        self.print_freq = print_freq
        vectorized_prompt = text_vectorization([prompt])[0].numpy()
        self.prompt_length = np.nonzero(vectorized_prompt == 0)[0][0]

    def on_epoch_end(self, epoch, logs=None):
        if (epoch + 1) % self.print_freq != 0:
            return
        for temperature in self.temperatures:
            print("== Generating with temperature", temperature)
            sentence = self.prompt
            for i in range(self.generate_length):
                tokenized_sentence = text_vectorization([sentence])
                predictions = self.model(tokenized_sentence)
                next_token = sample_next(
                    predictions[0, self.prompt_length - 1 + i, :]
                )
                sampled_token = tokens_index[next_token]
                sentence += " " + sampled_token
            print(sentence)

prompt = "This movie"
text_gen_callback = TextGenerator(
    prompt,
    generate_length=50,
    model_input_length=sequence_length,
    temperatures=(0.2, 0.5, 0.7, 1., 1.5))


model.fit(lm_dataset, epochs=200, callbacks=[text_gen_callback])

Found 100006 files belonging to 1 classes.
Epoch 1/200
391/391 [==============================] - ETA: 0s - loss: 5.9207== Generating with temperature 0.2
This movie is the evil babies finished watching that john a webb witch threads delves classical young of tall confession where 10 [UNK] the rest is a nice girl which is constructed by robert coproduction next rain arrives and on the constitution they can describe its spectacular summer jr and people then
== Generating with temperature 0.5
This movie is remarkably to learn over for the prince 1999 the great setting by [UNK] through and genuinely lane is somewhat damaging [UNK] length i get the music was so instantly girl of st and crew east [UNK] by jaws career thought that isnt wing this movie it in chemistry between
== Generating with temperature 0.7
This movie featuring ten highly dates which has making you read my first time a masterpiece i had a school fonda as willie for a little more believer and then they most veronica probabl

KeyboardInterrupt: 